In [ ]:
#Allow Colab notebook to access files on Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Necessary installations

!pip install openai
!pip install nlp

In [ ]:
#Access eli5 dataset

import nlp
eli5=nlp.load_dataset('eli5')

In [ ]:
#Save questions and highest-scored human answers

questions_arr=[]
best_answers_arr=[]
for i in range(len(eli5['train_eli5'])):
  q_dict=eli5['train_eli5'][i]
  questions_arr.append(q_dict['title'])
  best_ind=q_dict['answers']['score'].index(max(q_dict['answers']['score']))
  best_answers_arr.append(q_dict['answers']['text'][best_ind])


In [ ]:
#Query ChatGPT with eli5 questions and save outputs. This is written for computing all responses in eli5, but note that only a subset were used in this project due to compute time constraints.

import openai
import time

with open('/content/drive/MyDrive/eli5_chatgpt_responses_v2.txt','w') as f:
  openai.api_key='PUT KEY HERE'

  times_arr=[]
  start_time=time.time()
  count=0
  
  #Five queries are input at once, as this significantly increases computation speed.
  for i in range(len(questions_arr)//5):
    count+=1
    if count%100==0:
      print(count)
    output=openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Explain like I'm five: " + questions_arr[5*i]+'   Also, '+questions_arr[5*i+1]+'   Also, '+questions_arr[5*i+2]+'   Also, '+questions_arr[5*i+3]+'   Also, '+questions_arr[5*i+4]},

        ]
    )
    f.write(output['choices'][0]['message']['content'] + '\n\n\n\n')

    #The ChatGPT API limits unpaid users to three queries per minute
    times_arr.append(time.time()-start_time)
    if i<2:
      continue
    time_diff=times_arr[-1]-times_arr[-3]
    if time_diff<60:
      time.sleep(63-time_diff)
  

In [ ]:
#Save true responses to a text file

with open('/content/drive/MyDrive/CompLingFinalProject/josie_eli5_true_responses.txt','w') as f:
  for i in range(len(best_answers_arr)):
    f.write(best_answers_arr[i].replace('\n', ' ') + '\n')